View expermints of Mill-Turn-Zollern-Motor from database


Load Libraries

In [ ]:
#### Load Libraries
import numpy as np
from joblib import load
import glob 
import  pandas as pd
import matplotlib.pyplot as plt
import os
from pymongo import MongoClient
import plotly.express as ex
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
from datetime import timedelta
import plotly.io as pio

Set parameters

In [ ]:
workspace= "G:\\Innovations@HELLER\\DN\\KI\\Zollern-FH-MillTrunMotor\\Datasets\\rough_data\\database_with_sensor_synchronized\\Synchronization_results\\visualization"
###['Drehzahl','Temp_Motor','Temp_Klemmung', 'Temp_Lager']
output_variable='prediction'
nominated_signal= 'prediction'
prediction_variable= 'prediction'
time_shift= +1
selected_Columns= ['DRZ','Temp_Klemmung', 'Temp_Lager','Temp_Motor','Analytic_sol','ai_hmi_sol', prediction_variable  ]

Set time priod needed

In [ ]:
start= datetime.strptime("2024-01-18T00:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
end= datetime.strptime ("2024-01-20T16:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")


In [ ]:
host= "dnedge127v334"#"m53763edge"
port=27017
collection= "thermal_data"
all_data= True

Get Data from database

In [ ]:
## Read raw and preprocessing data
client = MongoClient(host= host, port=port)
db = client.h4ai
event_list = db[collection].find({ "date" : { '$gte' : start, '$lt' : end} }).sort('date', 1)
signals= None
i=0
last_prediction= -500
prev_time= None
current_time= None
stop= False
### Go through events in DB
for event in event_list:
    i+=1
    record_list= event['content']
    keys= None
    ### Go through records in each event
    for record in record_list:
        ### Get right data between start / end 
        if record['date']< start or record['date'] > end:
            continue
        current_time= record['date']+ timedelta(hours= time_shift)
        if prev_time is not None  and (current_time -prev_time).total_seconds()> 5.5:
            print('Period= ', (current_time -prev_time).total_seconds())
            print('prev_time', prev_time)
            print('current_time', current_time)
            print('#########################################################')
        for item in record['raw_data'].keys():
            record['raw_data'][item]= [np.double(np.round(record['raw_data'][item],decimals=2))]
        record['raw_data']['date']=[current_time]
        record['raw_data']['given2model']=[record['given2model']]
        record['raw_data']['prediction']= [record['prediction']]
        ####################################
        #print ('After ',record)
        lf_signal_point= pd.DataFrame(record['raw_data'])
        if signals is None:
            signals= lf_signal_point
        else:
            signals= signals.append(lf_signal_point,ignore_index= True)
        prev_time= current_time
signals.reset_index(inplace= True)
output=  signals[output_variable]
machine_prediction= signals[prediction_variable]

In [ ]:
scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'No.1,No.2,No.3,No.4,No.5,No.6,No.7,No.8
y_axis_names= selected_Columns
#print(preds)
fig= make_subplots(rows=5,cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02)
##DRZ
fig.add_trace(go.Scatter(x= signals['date'], y= signals['DRZ5'], name='given2model', mode= scatter_mode), row= 1, col= 1)
fig.update_yaxes(title_text= 'DRZ (RPM)', row= 1, col= 1)
##T_KLEMMUNG
fig.add_trace(go.Scatter(x= signals['date'], y= signals['T_KLEMMUNG'], name='T_KLEMMUNG', mode= scatter_mode), row= 2, col= 1)
fig.update_yaxes(title_text= 'T_KLEMMUNG C°', row= 2, col= 1)
##T_LAGER
fig.add_trace(go.Scatter(x= signals['date'], y= signals['T_LAGER'], name='T_LAGER', mode= scatter_mode), row= 3, col= 1)
fig.update_yaxes(title_text= 'T_LAGER C°', row= 3, col= 1)
##'T_MOTOR'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['T_MOTOR'], name='T_MOTOR', mode= scatter_mode), row= 4, col= 1)
fig.update_yaxes(title_text= 'T_MOTOR C°', row= 4, col= 1)
##'prediction_variable'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['ai_hmi_sol'], name='magnet_temperature ai_hmi_sol', mode= scatter_mode), row= 5, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= signals[prediction_variable], name=prediction_variable, mode= scatter_mode), row= 5, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= signals['Analytic_itc'], name='Analytic_itc', mode= scatter_mode), row= 5, col= 1)
fig.update_yaxes(title_text= 'Rotor Temperature C°', row= 5, col= 1)
##Restfehler
#fig.add_trace(go.Scatter(x= signals['date'], y= signals['No.8']- signals['Analytic_sol'], name=' Error-Analytic', mode= scatter_mode),  row= 6 , col= 1)
#fig.add_trace(go.Scatter(x= signals['date'], y= signals['No.8']- signals[prediction_variable], name=' Error-Data-Driven', mode= scatter_mode),  row= 6 , col= 1)
    ## Draw the tolerence +-5
#fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals['db_prediction_abs_error'],5), name='+5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
#fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals['db_prediction_abs_error'],-5), name='- 5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
fig.update_yaxes(title_text= 'Error in Rotor Temperature C°', row= 6, col= 1)
fig.update_layout(height=800, width=1400, title_text= 'M57002 Machine Data')
fig.show()
        

In [ ]:
# signals.to_csv('Dauerlauf12_13_10_2023.csv')